In [9]:
import requests
import pandas as pd

In [2]:
query="seo" #Define Your Query
url = f"https://api.pushshift.io/reddit/search/comment/?q={query}"
request = requests.get(url)
json_response = request.json()
json_response

{'data': [{'all_awardings': [],
   'associated_award': None,
   'author': 'karldemort',
   'author_flair_background_color': '',
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_template_id': None,
   'author_flair_text': None,
   'author_flair_text_color': '',
   'author_flair_type': 'text',
   'author_fullname': 't2_17321nb9',
   'author_patreon_flair': False,
   'author_premium': False,
   'awarders': [],
   'body': 'Ok shipping Han Ji Pyeong and Nam Do San already. Sorry Seo Dal Mi',
   'collapsed_because_crowd_control': None,
   'comment_type': None,
   'created_utc': 1604241384,
   'gildings': {},
   'id': 'gasn8fa',
   'is_submitter': False,
   'link_id': 't3_jlynuz',
   'locked': False,
   'no_follow': True,
   'parent_id': 't3_jlynuz',
   'permalink': '/r/KDRAMA/comments/jlynuz/startup_episode_6/gasn8fa/',
   'retrieved_on': 1604241395,
   'score': 1,
   'send_replies': True,
   'stickied': False,
   'subreddit': 'KDRAMA',
   'subreddit_id': '

In [3]:
def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [4]:
data_type="comment"     # give me comments, use "submission" to publish something
query="python"          # Add your query
duration="30d"          # Select the timeframe. Epoch value or Integer + "s,m,h,d" (i.e. "second", "minute", "hour", "day")
size=1000               # maximum 1000 comments
sort_type="score"       # Sort by score (Accepted: "score", "num_comments", "created_utc")
sort="desc"             # sort descending
aggs="subreddit"        #"author", "link_id", "created_utc", "subreddit"

In [5]:
get_pushshift_data(data_type=data_type,     
                   q=query,                 
                   after=duration,          
                   size=size,               
                   sort_type=sort_type,
                   sort=sort)

{'data': [{'all_awardings': [{'award_sub_type': 'GLOBAL',
     'award_type': 'global',
     'awardings_required_to_grant_benefits': None,
     'coin_price': 300,
     'coin_reward': 0,
     'count': 2,
     'days_of_drip_extension': 0,
     'days_of_premium': 0,
     'description': "When an upvote just isn't enough, smash the Rocket Like.",
     'end_date': None,
     'giver_coin_reward': None,
     'icon_format': None,
     'icon_height': 2048,
     'icon_url': 'https://i.redd.it/award_images/t5_22cerq/94pn64yuas941_RocketLike.png',
     'icon_width': 2048,
     'id': 'award_28e8196b-d4e9-45bc-b612-cd4c7d3ed4b3',
     'is_enabled': True,
     'is_new': False,
     'name': 'Rocket Like',
     'penny_donate': None,
     'penny_price': None,
     'resized_icons': [{'height': 16,
       'url': 'https://preview.redd.it/award_images/t5_22cerq/94pn64yuas941_RocketLike.png?width=16&amp;height=16&amp;auto=webp&amp;s=c1400eb6ea235d0c96c3aa6e271c71d7f339cbd4',
       'width': 16},
      {'height

In [6]:
data = get_pushshift_data(data_type=data_type,
                          q=query,
                          after=duration,
                          size=size,
                          aggs=aggs)

In [7]:
data = data.get("aggs").get(aggs)

In [10]:
df = pd.DataFrame.from_records(data)[0:10]
df

,doc_count,key
0,5444,wrns
1,4387,learnpython
2,3014,Python
3,2855,hentai
4,2024,ProgrammerHumor
5,1913,learnprogramming
6,1082,AskReddit
7,746,EliteDangerous
8,735,datascience
9,716,ecchi


In [27]:
# Call the API
data = get_pushshift_data(data_type=data_type,
                          q=query,
                          after="7d",
                          size=10,
                          sort_type=sort_type,
                          sort=sort).get("data")
 

In [ ]:
data

In [19]:
# Select the columns you care about
df = pd.DataFrame.from_records(data)[["author", "subreddit", "score", "body", "permalink"]]

# Keep the first 400 characters
df['body'] = df['body'].str[0:400] + "..."
 
# Append the string to all the permalink entries so that we have a link to the comment
df['permalink'] = "https://reddit.com" + df['permalink'].astype(str)
 
 
# Create a function to make the link to be clickable and style the last column
def make_clickable(val):
    """ Makes a pandas column clickable by wrapping it in some html.
    """
    return '<a href="{}">Link</a>'.format(val,val)
 
df.style.format({'permalink': make_clickable})

,author,subreddit,score,body,permalink
0,rishav_sharan,india,1,"Here is some generic stuff that i would advise to anyone getting into IT; * unless the company is super good or super bad, change it after every 2 years. always ask for at least 25% over the last job. * Always keep an active job profile, even if you are not actively looking. * Try to get into a startup in your 2nd job (the experience will count a lot later, when you are going for senior roles and...",Link
1,The_Procrastinator10,Discord_Bots,1,python...,Link
2,FreeWildbahn,AskProgramming,1,I also started with pycharm community edition. I really liked. Especially if you just start with python. Without any hassle you have good autocompletion and a very nice linter. The linter helps you to develop a good style. I switched to VS Code just because it has the better vim mode. But it takes more time to configure it....,Link
3,CoffeePython,Python,1,Sort of a plug: I built [usepandas.com](https://www.usepandas.com) this week and have been working on adding more pandas recipes. It’s an open-source collection of easy to use Python pandas recipes! I even got our first user submitted recipe this morning! It feels good making something useful for the community and I’m excited to see what other recipes people submit....,Link
4,nam_148,datascience,1,"Yes the z value, but I was looking for an explanation of how you would do it using python. But still, thankyou!...",Link
5,clarinetist001,datascience,1,"This reminds me of my actuarial science days. I don't miss them. If you haven't learned how to automate such processes using something like Python or R, I would suggest you learn how to do that. However, I don't know of a single course that would guide you through the process of taking an Excel spreadsheet that could be formatted any number of ways and gives the data a cleaner structure. Excel a...",Link
6,chellu,AskReddit,1,"Then Gandalf the Grey and Gandalf the White, and Monty Python and the Holy Grail's Black Knight, and Benito Mussolini, and the Blue Meanie, and Cowboy Curtis and Jambi the Genie, Robocop, Terminator, Captain Kirk and Darth Vader, Lo Pan, Super Man, every single Power Ranger, Bill S Preston and Theodore Logan, Spock, The Rock, Doc Ock and Hulk Hogan, all came out of nowhere lightning fast, and they...",Link
7,4dam_Kadm0n,qutebrowser,1,"No I totally get that, but the autoconfig yaml file is just not the place to do that (I don't know exactly why, TBH). You can use "":config-write-py"" to generate a python config file. You can then edit this config.py file to your heart's content - it has more options than the autoconfig file, too. Qutebrowser will always give the config.py file precedence over the autoconfig file if both are pres...",Link
8,AutoModerator,Python,1,"Hi there, from the /r/Python mods. We have removed this post as it is not suited to the /r/Python subreddit proper, however it should be very appropriate for our sister subreddit /r/LearnPython or for the r/Python discord: https://discord.gg/python. The reason for the removal is that /r/Python is dedicated to discussion of Python news, projects, uses and debates. It is not designed to act as Q&a...",Link
9,Shady_Bloke,pics,1,"I created this I decided to make a word cloud of the presidential debate that occurred on 10-22-2020 between Joe Biden and Donald Trump. This word cloud has many commonly used words removed including “of”, “the”, “and”, etc. The process to create this image involved the wordcloud python library, python v3.7.6, and GIMP. Full credit for the silhouette and background images goes to the link below. ...",Link
